# Imports

In [30]:
import  tensorflow                  as      tf
import  numpy                       as      np
import  os
from    sklearn.model_selection     import  train_test_split
from    tensorflow.keras.utils      import  to_categorical

from    tensorflow.keras.models     import  Sequential
from    tensorflow.keras.layers     import  LSTM, Dense
from    tensorflow.keras.callbacks  import  TensorBoard

## Constants

In [31]:
SAMPLES_PER_WORD = 60
FRAMES_PER_SAMPLE = 30

SAMPLES_PATH = "/Volumes/SSD/collection/"

In [32]:

words = np.array(['none', 'person', 'hello', 'i love you', 'thank you', 'you', 'meet', 'nice', 'us', 'listening'])
words_path = {
    'hello' : 59,
    'none' : 61,
    'person' : 63,
    'i love you' : 75,
    'thank you' : 71,
    'you' : 76,
    'meet' : 73,
    'nice' : 74,
    'us' : 77,
    'listening' : 78
    }

labels_map = {label:num for num, label in enumerate(words)}
labels_map

{'none': 0,
 'person': 1,
 'hello': 2,
 'i love you': 3,
 'thank you': 4,
 'you': 5,
 'meet': 6,
 'nice': 7,
 'us': 8,
 'listening': 9}

In [33]:

samples, labels = [], []
for word in words:
    for sample in range(SAMPLES_PER_WORD):
        sample_data = []
        for frame in range(FRAMES_PER_SAMPLE):
            path        = os.path.join(SAMPLES_PATH, f'{words_path[word]}', f'{words_path[word]}_{sample}_{frame}.npy')
            frame_data  = np.load(path)
            sample_data.append(frame_data)
        samples.append(sample_data)
        labels.append(labels_map[word])


In [34]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [21]:
X = np.array(samples)
X.shape

(480, 30, 225)

In [28]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model           = Sequential()

hands   = 63 + 63
pose    = 99

lstm_model_1    = LSTM(     64,     return_sequences = True,    activation = "relu", input_shape=(30, hands+pose) )
lstm_model_2    = LSTM(     128,    return_sequences = True,    activation = "relu" )
lstm_model_3    = LSTM(     64,     return_sequences = False,   activation = "relu" )

dense_model_1   = Dense(    64,                                 activation = "relu" )
dense_model_2   = Dense(    32,                                 activation = "relu" )
dense_model_3   = Dense(    words.shape[0],                     activation = "softmax" )

model.add( lstm_model_1 )
model.add( lstm_model_2 )
model.add( lstm_model_3 )

model.add( dense_model_1 )
model.add( dense_model_2 )
model.add( dense_model_3 )

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - categorical_accuracy: 0.1497 - loss: 2.1279
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.2895 - loss: 1.8687
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.3448 - loss: 1.7979
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.3010 - loss: 1.6098
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.3935 - loss: 1.6877
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.4996 - loss: 1.3942
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.5163 - loss: 1.2855
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - categorical_accuracy: 0.6444 - loss: 1.0133
Epoch 9/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.6870 - loss: 0.7898
Epoch 10/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - categorical_accuracy: 0.6944 - loss: 0.8309
Epoch 11/

In [27]:
model.save('new4.keras')